In [ ]:
import sys
sys.path.append('../src')

from time_series import (
    load_data,
    generate_lags,
    generate_lagged_df,
    split_df,
    fit_model,
    predict_evaluate_model,
)


In [ ]:
df = load_data('../data/data_original_m6.csv')
df.head(3)

In [ ]:
# Load dataset
y = df['ABBV']

n_lags_future = 2
lags_past, lags_future, lags_ar, lags_arp = generate_lags(n_lags_past=2*n_lags_future, n_lags_future=n_lags_future)
print(lags_past)
print(lags_future)
print(lags_ar)
print(lags_arp)


In [ ]:
df_lagged = generate_lagged_df(
    series=y,
    n_lags_past=2*n_lags_future,
    n_lags_future=n_lags_future,
)

display(df_lagged)

In [ ]:
df_train, df_test = split_df(df_lagged, train_test_ratio=0.8)
display(df_train)
display(df_test.head())


In [ ]:
from sklearn.linear_model import LinearRegression

model = fit_model(df_train,
                  features=lags_arp,
                  model_class=LinearRegression,
                  )


In [ ]:
# Predict and evaluate on Training set
y_pred_train, mape_train = predict_evaluate_model(model,
                                      df_train,
                                      features=lags_arp
                                      )

display(y_pred_train.head())
print(f"MAPE: {mape_train:.4f}")


In [ ]:
# Predict and evaluate on Testing set
y_pred_test, mape_test = predict_evaluate_model(model,
                                      df_test,
                                      features=lags_arp
                                      )

display(y_pred_test.head())
print(f"MAPE: {mape_test:.4f}")
